# Machine Learning Enabled Visualization
## Variational Autoencoders and Latent Space Representations of Astronomical Data
### Bryan Scott, CIERA | Northwestern

This tutorial but has been adapted here from one by Charles Kenneth Fisher and Raghav Kansal. That tutorial was itself an adaptation of a great introduction to VAEs by Louis Tiao. The solutions notebook will include links to this heritage.

This example should be taken mostly as a framework/outline of how to build an autoencoder. In a research setting, careful choices about loss functions and architecture should be investigated. Recent work, for example, on (non-variational) autoencoders has shown the importance of accounting for *redshift invariance* in galaxy spectra. Since the focus here is getting some practice with ML enabled visualization, these deeper questions are left for a future version of this tutorial.



Install specific version of tensorflow
(to simplify with some syntax changes in more recent versions)

In [3]:
pip install tensorflow==2.14.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 489.8/489.8 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 39.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 91.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.7/440.7 kB 26.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.6 MB/s eta 0:00:00
  Attempting uninstall: wrapt
    Found existing installation: wrapt 1.16.0
    Uninstalling wrapt-1.16.0:
      Successfully uninstalled wrapt-1.16.0
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-dtypes 0.4.1
    Uninstalling ml-dtypes-0.4.1:
      Successfully uninstalled ml-dtypes-0.4.1
  Attempting uninstall: keras
    Found existing installation: keras 3.4.1
    Uninstalling keras-3.4.1:
      Successfully uninstalled keras-3.4.1
  Attempting uninstall: google-auth-oauthlib
    Found existing ins

Loading required libraries, numpy, matplotlib, and a scipy.norm
(which we will use for visualizing a latent space with assumed gaussian distribution)

This tutorial is in keras which is a convenient library for abstracting Deep Learning code and making things more readable. The model itself will be trained in tensorflow. In Session 19, we used pytorch to teach Deep Learning, and it would be nice to convert this tutorial to pytorch (hack day project?).

In [13]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm

from tensorflow.keras import backend as K

from tensorflow.keras.layers import Input, InputLayer, Dense, Lambda, Layer, Add, Multiply
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.datasets import mnist
import tensorflow.compat.v1 as tf
import pandas as pd

Some lines to read in the Galaxy Zoo 2 Hubble Space Telescope Images. These have been downsized to 128x128 to keep this within colab's RAM limits. 14000 total images are provided in the GZ2 HST dataset, of which we will train on ~8000. You should play around with the number of images to train on.

In [21]:
ls /content/drive/MyDrive/DSFP_galaxy_zoo/

images/  resized_images/


In [4]:
import os

In [5]:
for filename in os.listdir("/content/drive/MyDrive/DSFP_galaxy_zoo/images")[0:1]:
  print("/content/drive/MyDrive/DSFP_galaxy_zoo/resized_images"+filename)

/content/drive/MyDrive/DSFP_galaxy_zoo/resized_images114435.jpg


In [22]:
from skimage.transform import resize
import imageio

import os

x_train = []

for filename in os.listdir("/content/drive/MyDrive/DSFP_galaxy_zoo/images")[0:8000]:
  print(filename)
  img = imageio.imread(filename)
  img_resize = resize(img, (128, 128))
  img_resize = (img_resize * 255).astype(np.uint8) #convert to int
  imageio.imwrite("/content/drive/MyDrive/DSFP_galaxy_zoo/resized_images/"+filename, img_resize)
  # x_train.append(img_resize)

114435.jpg
114437.jpg
114428.jpg
114430.jpg
114424.jpg
114420.jpg


<ipython-input-22-6b4d764ec98f>:10: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  img = imageio.imread(filename)


Streaming output truncated to the last 5000 lines.
11147.jpg
111481.jpg
111475.jpg
111486.jpg
111480.jpg
111474.jpg
11149.jpg
111483.jpg
111467.jpg
111505.jpg
111492.jpg
111494.jpg
111506.jpg
111501.jpg
111499.jpg
111493.jpg
111495.jpg
111497.jpg
111508.jpg
111498.jpg
111490.jpg
111491.jpg
111500.jpg
111496.jpg
111503.jpg
1115.jpg
11150.jpg
111502.jpg
111520.jpg
111515.jpg
11151.jpg
111526.jpg
111510.jpg
111521.jpg
111535.jpg
111509.jpg
111517.jpg
111531.jpg
111536.jpg
111527.jpg
111512.jpg
111524.jpg
11152.jpg
111525.jpg
111532.jpg
111533.jpg
111511.jpg
111523.jpg
111516.jpg
111519.jpg
111534.jpg
111549.jpg
111540.jpg
111539.jpg
111538.jpg
111562.jpg
111558.jpg
111557.jpg
111551.jpg
111552.jpg
111563.jpg
111555.jpg
111548.jpg
111556.jpg
111550.jpg
111543.jpg
111541.jpg
111561.jpg
111559.jpg
11154.jpg
111554.jpg
111547.jpg
111545.jpg
111565.jpg
111580.jpg
111566.jpg
111583.jpg
111584.jpg
11157.jpg
111573.jpg
111587.jpg
111569.jpg
111582.jpg
111568.jpg
111567.jpg
111585.jpg
111564.jpg
1

Problem 1: Look at the last layer of the model to answer the following question - why do the HST images get divided by 255.0?

In [10]:
x_train = np.array(x_train) / 255.0
print(x_train.shape)

(4000, 128, 128, 3)


Problem 2: Visualize a few of the input images.

In [ ]:
# visualization code goes here

The following cell defines some dimensions we choose and gets the shape of the data for later required reshapes.

In [ ]:
# Find dimensions of input images
img_rows, img_cols, img_chns = x_train.shape[1:]

# Specify hyperparameters
original_dim = img_rows * img_cols
intermediate_dim = 256
latent_dim = 2
batch_size = 100
epochs = 10
epsilon_std = 1.0

The following cell defines the loss function. There are two terms, the binary cross entropy between the inputs and outputs, and a KL divergence layer which 'regularizes' such that the distribution of latent space vectors approximates a Gaussian.

In [ ]:
def nll(y_true, y_pred):
    """Negative log likelihood (Bernoulli)."""

    # keras.losses.binary_crossentropy gives the mean
    # over the last axis. we require the sum
    return K.sum(K.binary_crossentropy(y_true, y_pred), axis=-1)


class KLDivergenceLayer(Layer):
    """Identity transform layer that adds KL divergence
    to the final model loss.
    """

    def __init__(self, *args, **kwargs):
        self.is_placeholder = True
        super(KLDivergenceLayer, self).__init__(*args, **kwargs)

    def call(self, inputs):
        mu, log_var = inputs

        kl_batch = -0.5 * K.sum(1 + log_var - K.square(mu) - K.exp(log_var), axis=-1)

        self.add_loss(K.mean(kl_batch))

        return inputs

This defines the encoder and decoder layers. The lines involving z are telling the model that the latent space distribution should be a Gaussian, while the lines with eps are the 'reparameterization trick' that we'll cover in a future session.

The encoder takes data and maps it to a distribution of latent space vectors. The decoder takes vectors on the latent space and maps them back to the data.

Problem 3: Make sure you understand (roughly) what each line here does, and check that you know why each layer has the dimensions that they do.

In [ ]:
# Encoder

x = Input(shape=(original_dim,))
h = Dense(intermediate_dim, activation="relu")(x)

z_mu = Dense(latent_dim)(h)
z_log_var = Dense(latent_dim)(h)

z_mu, z_log_var = KLDivergenceLayer()([z_mu, z_log_var])

# Reparametrization trick
z_sigma = Lambda(lambda t: K.exp(0.5 * t))(z_log_var)

eps = Input(tensor=K.random_normal(shape=(K.shape(x)[0], latent_dim))) # note that this line changes in newer keras/tensorflow versions

z_eps = Multiply()([z_sigma, eps])
z = Add()([z_mu, z_eps])

# This defines the Encoder which takes noise and input and outputs
# the latent variable z
encoder = Model(inputs=[x, eps], outputs=z)

# Decoder is MLP specified as single Keras Sequential Layer
decoder = Sequential(
    [
        Dense(intermediate_dim, input_dim=latent_dim, activation="relu"),
        Dense(original_dim, activation="sigmoid"),
    ]
)

x_pred = decoder(z)

Define the Variational Autoencoder

In [ ]:
vae = Model(inputs=[x, eps], outputs=x_pred, name="vae")
vae.compile(optimizer="rmsprop", loss=nll)
vae.summary()

Train the Model

In [ ]:
x_train = x_train.reshape(-1, original_dim)

hist = vae.fit(
    x_train,
    x_train,
    shuffle=True,
    epochs=epochs,
    batch_size=batch_size)

Problem 4: Plot the latent space vectors. Since the latent space dimension is 2D, these will just be points in a plane. This is dangerous - since it may not, and we are pattern seeking creatures who find them when none exist - but our goal is to see if the latent space has some clear structure we can interpret.

In [ ]:
x_train_encoded = encoder.predict(x_train, batch_size=batch_size)
plt.figure()
plt.scatter(x_train_encoded[:, 0], x_train_encoded[:, 1])
plt.colorbar()
plt.show()

Problem 5: Now, create a grid of images where each image is a uniform sample from the latent space. In other words, uniformly sample vectors on the 2D latent space and use these as inputs to the decoder, which will give you an image back. The grid you produce here should have the same structure as the latent space.

In [ ]:
  # display a 2D manifold of the images
n = 5  # figure with 15x15 images
quantile_min = 0.01
quantile_max = 0.99

# Linear Sampling
# we will sample n points within [-15, 15] standard deviations
z1_u =
z2_u =
z_grid = np.dstack()

x_pred_grid = decoder.predict().reshape() #hint, should take latent space vectors and reshape them to make a figure grid


ax.set_xlabel("$z_1$")
ax.set_ylabel("$z_2$")
ax.set_title("Uniform")

Problem 5: We assumed that the latent space was a Gaussian. So instead of sampling uniformly, now sample from latent space such that the probability of selecting each point is given by the gaussian distribution on the latent space. Visualize these.

In [ ]:
# Inverse CDF sampling
# hint, same as before, but you might find norm.ppf helpful.
# You should have a sense of why specifically this function is helpful
z1 =
z2 =
z_grid2 = np.dstack()

x_pred_grid2 = decoder.predict().reshape()



ax.set_xlabel("$z_1$")
ax.set_ylabel("$z_2$")
ax.set_title("Inverse CDF")

Problem 6: After completing the visualization steps above, change the latent space dimension to N > 3. Use a dimensionality reduction or clustering technique to visualize the latent space.

How stable is the latent space representation that you have learned? Does it change if the image dimensions change? What about your interpretation?